In [1]:
class Lan(object):
    def __init__(self, key):
        self.key = key
        self.bridges = {}
        self.hosts = {}
        
        self.msgs = []
        self.db = None
        self.t = 0

    def add_bridge(self, bridge):               # Add Bridge IDs in Lan
        self.bridges[bridge.key] = bridge
        
    def add_host(self, host):                   # Add Hosts in Lan
        self.hosts[host] = self.key
        
    def send_msg(self):                         # Get Bridge IDs in Lan
        for msg in self.msgs:
            state = msg[0]
            sender = msg[1]
            for bridge in self.bridges:
                if(sender != bridge):
                    self.bridges[bridge].recieve_msg([state, self.key], self.t)
    
    def recieve_msg(self, msg, t):                  # Get Bridge IDs in Lan
        self.t = t
        self.msgs.append(msg)
    
    def transmit(self, sender, reciever, bridge_key):
        if bridge_key is None:
            for bridge in self.bridges:
                 if not (self.bridges[bridge].is_null(self.key)):
                    print(sender," -->", bridge)
                    self.bridges[bridge].transmit(sender,reciever,self.key)
            return
            
        else:
            for bridge in self.bridges:
                if not (self.bridges[bridge].is_null(self.key) or bridge == bridge_key):
                    print(sender," -->", bridge)
                    self.bridges[bridge].transmit(sender, reciever, self.key)
            return

        return
        
    def update(self):
        self.msgs.sort(key=order)
        
        if len(self.msgs) != 0:
            best_config = self.msgs[0]
            bridge_key = best_config[1]
            
            if self.db is not None:
                if (bridge_key[1] < self.db[1]):
                    self.db = bridge_key
            else:
                self.db = bridge_key
                
        self.msgs = []
        return self.db
        
    def get_connections(self):                  # Get Bridge IDs in Lan
        return self.bridges.keys()
    
    def get_hosts(self):                        # Get Host IDs in Lan
        return self.hosts.keys()

In [2]:
class Bridge(object):
    def __init__(self, key):
        self.key = key    
        self.root = key
        self.d = 0
        
        self.lans = {}
        self.state = [self.root, self.d, self.key]
        self.msgs = []
        
        self.trace = []
        self.change = False
        
        self.forward = {}

    def add_lan(self, lan):                     # Add Lan in Bridge
        self.lans[lan.key] = [lan,"DP"]

    def get_connections(self):                  # Get Lan IDs in Bridge
        return self.lans.keys()
    
    def port(self, lan_key):                    # Get Lan IDs in Bridge
        return [self.key, lan_key, self.lans[lan_key][1]]
    
    def status(self):                           # Get Bridge Status
        return self.state
    
    def send_msg(self, t):
        self.trace.append(str(t) + " s " + self.key + " (" +self.root+", " + str(self.d)+", " +self.key + ")")
        for lan in self.lans:
            self.lans[lan][0].recieve_msg([self.state, bridge], t)

    def recieve_msg(self, msg, t):                      # Receiver Bridge Status
        self.trace.append(str(t+1)+" r " +self.key+" (" + msg[0][0]+", " + str(msg[0][1])+", " + msg[0][2]+")")
        self.msgs.append(msg)
    
    def transmit(self, sender, reciever, lan_key):
        print(self.forward)
        if reciever not in self.forward:
            self.forward[sender] = lan_key
            
            for lan in self.lans:
                if not (lan == lan_key or self.is_null(lan)):
                    print(sender," -->", lan)
                    self.lans[lan][0].transmit(sender, reciever, self.key)
            return
        
        else:
            if sender not in self.forward:
                self.forward[sender] = lan_key
                
                lan_key = self.forward[reciever]
                print(sender," -->", lan_key)
                self.lans[lan_key][0].transmit(sender, reciever, self.key)
                
                return
            
            else:
                lan_key = self.forward[reciever]
                print(sender," -->", lan_key)
                self.lans[lan_key][0].transmit(sender, reciever, self.key)
                
                return
                
    def update(self):
        self.msgs.sort(key=order)
        
        if len(self.msgs) != 0:
            best_config = self.msgs[0]
            
            if(self.root > best_config[0][0]):
                self.root = best_config[0][0]
                self.d = int(best_config[0][1]) + 1

                lan_key = best_config[1]
                self.lans[lan_key][1] = 'RP'
                
                self.state = [self.root, self.d, self.key]
                self.change = True
                
            else:
                self.change = False
            
        self.msgs = []    
        return self.state
    
    def null_port(self):
        for lan in self.lans:
            if self.lans[lan][1] != 'RP':
                if self.lans[lan][0].db != self.key:
                    self.lans[lan][1] = 'NP'
    
    def is_null(self,lan_key):
        return self.lans[lan_key][1] == 'NP'
        
    def is_root(self):
        return self.root == self.key

In [3]:
def order(msg):
    return int(msg[0][0][1])**2 + int(msg[0][1])**2 + int(msg[0][2][1])**2

In [4]:
def torder(t):
    return t[0]

In [5]:
class Network(object):
    def __init__(self):
        self.bridges = {}   # Bridges in Network
        self.lans = {}      # Lans in Network
        self.hosts = {}
        self.update = True
        
    def add_bridge(self, bridge):                   # Add Bridge in Network
        self.bridges[bridge.key] = bridge
        
    def add_lan(self, lan):                         # Add Lan in Network
        self.lans[lan.key] = lan
        
    def add_host(self, host, lan):                  # Add Host in Network
        self.hosts[host] = lan
        self.lans[lan].hosts[host] = lan
        
    def add_port(self, lan_key, bridge_key):        # Add Port in resp Bridge and Lan
        if lan_key not in self.lans:
            self.add_lan(Lan(lan_key))
            
        if lan_key not in self.bridges[bridge_key].lans:
            self.bridges[bridge_key].add_lan(self.lans[lan_key])
            
        if bridge_key not in self.lans[lan_key].bridges:
            self.lans[lan_key].add_bridge(self.bridges[bridge_key])
    
    def transmit(self, sender, reciever):
        lan_key = self.hosts[sender]
        self.lans[lan_key].transmit(sender,reciever, None)
        
        return
        
    def get_bridges(self):                          # Get Bridges IDs in Network
        return self.bridges.keys()
    
    def get_lans(self):                             # Get Lans IDs in Network
        return self.lans.keys()
    
    def change(self):
        for bridge in self.bridges:
            if self.bridges[bridge].change == True:
                self.update = True
                break
                
        return self.update
    
    def __iter__(self):
        return iter(self.bridges.values())

In [6]:
net = Network()

In [7]:
net.add_bridge(Bridge('B1'))
net.add_port('A','B1')
net.add_port('G','B1')
net.add_port('B','B1')

In [8]:
net.add_bridge(Bridge('B2'))
net.add_port('G','B2')
net.add_port('F','B2')

In [9]:
net.add_bridge(Bridge('B3'))
net.add_port('B','B3')
net.add_port('C','B3')

In [10]:
net.add_bridge(Bridge('B4'))
net.add_port('C','B4')
net.add_port('F','B4')
net.add_port('E','B4')

In [11]:
net.add_bridge(Bridge('B5'))
net.add_port('C','B5')
net.add_port('D','B5')
net.add_port('E','B5')

In [12]:
for bridge in net.bridges:
    print(net.bridges[bridge].status())

['B1', 0, 'B1']
['B2', 0, 'B2']
['B3', 0, 'B3']
['B4', 0, 'B4']
['B5', 0, 'B5']


In [13]:
flag = 1
t = 0
trace = []
while net.update:

    net.update = False
    
    for bridge in net.bridges:
        net.bridges[bridge].send_msg(t)

        for lan in net.bridges[bridge].lans:
            net.bridges[bridge].lans[lan][0].send_msg()
            net.bridges[bridge].lans[lan][0].update()

        net.bridges[bridge].update()
        trace.append(net.bridges[bridge].trace)
        
    t += 1
    net.change()
    
for bridge in net.bridges:
    net.bridges[bridge].null_port()

if(flag):
    for t in trace:
        t.sort(key=torder)
        print(t)

['0 s B1 (B1, 0, B1)', '1 r B1 (B2, 0, B2)', '1 r B1 (B3, 0, B3)', '1 s B1 (B1, 0, B1)', '2 r B1 (B1, 1, B2)', '2 r B1 (B1, 1, B3)', '2 s B1 (B1, 0, B1)', '3 r B1 (B1, 1, B2)', '3 r B1 (B1, 1, B3)']
['0 s B2 (B2, 0, B2)', '1 r B2 (B1, 0, B1)', '1 r B2 (B4, 0, B4)', '1 s B2 (B1, 1, B2)', '2 r B2 (B1, 0, B1)', '2 r B2 (B2, 1, B4)', '2 s B2 (B1, 1, B2)', '3 r B2 (B1, 0, B1)', '3 r B2 (B1, 2, B4)']
['0 s B3 (B3, 0, B3)', '1 r B3 (B1, 0, B1)', '1 r B3 (B4, 0, B4)', '1 r B3 (B5, 0, B5)', '1 s B3 (B1, 1, B3)', '2 r B3 (B1, 0, B1)', '2 r B3 (B2, 1, B4)', '2 r B3 (B3, 1, B5)', '2 s B3 (B1, 1, B3)', '3 r B3 (B1, 0, B1)', '3 r B3 (B1, 2, B4)', '3 r B3 (B1, 2, B5)']
['0 s B4 (B4, 0, B4)', '1 r B4 (B2, 0, B2)', '1 r B4 (B3, 0, B3)', '1 r B4 (B5, 0, B5)', '1 r B4 (B5, 0, B5)', '1 s B4 (B2, 1, B4)', '2 r B4 (B1, 1, B2)', '2 r B4 (B1, 1, B3)', '2 r B4 (B3, 1, B5)', '2 r B4 (B3, 1, B5)', '2 s B4 (B1, 2, B4)', '3 r B4 (B1, 1, B2)', '3 r B4 (B1, 1, B3)', '3 r B4 (B1, 2, B5)', '3 r B4 (B1, 2, B5)']
['0 s 

In [14]:
print(net.bridges['B1'].msgs)

[[['B1', 1, 'B2'], 'G'], [['B1', 1, 'B3'], 'B']]


In [15]:
for bridge in net.bridges:
    for lan in net.bridges[bridge].lans:
        print(net.bridges[bridge].port(lan))

['B1', 'A', 'DP']
['B1', 'G', 'DP']
['B1', 'B', 'DP']
['B2', 'G', 'RP']
['B2', 'F', 'DP']
['B3', 'B', 'RP']
['B3', 'C', 'DP']
['B4', 'C', 'NP']
['B4', 'F', 'RP']
['B4', 'E', 'DP']
['B5', 'C', 'RP']
['B5', 'D', 'DP']
['B5', 'E', 'NP']


In [16]:
for bridge in net.bridges:
    for lan in net.bridges[bridge].lans:
        print(net.bridges[bridge].port(lan))

['B1', 'A', 'DP']
['B1', 'G', 'DP']
['B1', 'B', 'DP']
['B2', 'G', 'RP']
['B2', 'F', 'DP']
['B3', 'B', 'RP']
['B3', 'C', 'DP']
['B4', 'C', 'NP']
['B4', 'F', 'RP']
['B4', 'E', 'DP']
['B5', 'C', 'RP']
['B5', 'D', 'DP']
['B5', 'E', 'NP']


In [17]:
print(net.bridges['B4'].state)

['B1', 2, 'B4']


In [18]:
print(net.lans['E'].db)

B4


In [19]:
print(net.lans['G'].bridges)

{'B1': <__main__.Bridge object at 0x7fe1947b5828>, 'B2': <__main__.Bridge object at 0x7fe1947b5e10>}


In [20]:
net.add_host('H1','A')
net.add_host('H2','A')
net.add_host('H3','A')

In [21]:
net.add_host('H4','B')
net.add_host('H5','B')

In [22]:
net.add_host('H6','C')
net.add_host('H7','C')
net.add_host('H8','C')

In [23]:
net.add_host('H9','D')
net.add_host('H10','D')

In [24]:
net.add_host('H11','E')

In [25]:
net.add_host('H12','F')
net.add_host('H13','F')

In [26]:
net.add_host('H14','G')

In [27]:
net.transmit('H9','H2')

H9  --> B5
{}
H9  --> C
H9  --> B3
{}
H9  --> B
H9  --> B1
{}
H9  --> A
H9  --> G
H9  --> B2
{}
H9  --> F
H9  --> B4
{}
H9  --> E


In [28]:
for bridge in net.bridges:
    print(net.bridges[bridge].forward)

{'H9': 'B'}
{'H9': 'G'}
{'H9': 'C'}
{'H9': 'F'}
{'H9': 'D'}


In [29]:
net.transmit('H4','H12')

H4  --> B1
{'H9': 'B'}
H4  --> A
H4  --> G
H4  --> B2
{'H9': 'G'}
H4  --> F
H4  --> B4
{'H9': 'F'}
H4  --> E
H4  --> B3
{'H9': 'C'}
H4  --> C
H4  --> B5
{'H9': 'D'}
H4  --> D


In [30]:
for bridge in net.bridges:
    print(net.bridges[bridge].forward)

{'H9': 'B', 'H4': 'B'}
{'H9': 'G', 'H4': 'G'}
{'H9': 'C', 'H4': 'B'}
{'H9': 'F', 'H4': 'F'}
{'H9': 'D', 'H4': 'C'}


In [31]:
net.transmit('H3','H9')

H3  --> B1
{'H9': 'B', 'H4': 'B'}
H3  --> B
H3  --> B3
{'H9': 'C', 'H4': 'B'}
H3  --> C
H3  --> B5
{'H9': 'D', 'H4': 'C'}
H3  --> D


In [32]:
for bridge in net.bridges:
    print(net.bridges[bridge].forward)

{'H9': 'B', 'H4': 'B', 'H3': 'A'}
{'H9': 'G', 'H4': 'G'}
{'H9': 'C', 'H4': 'B', 'H3': 'B'}
{'H9': 'F', 'H4': 'F'}
{'H9': 'D', 'H4': 'C', 'H3': 'C'}
